In [6]:
import os
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

import sistemas
import sei
import page

In [4]:
usr = "rsilva"
pwd = "Savorthemom3nts"

In [ ]:
#sei_browser = sei.login_sei(driver=sei.webdriver.Firefox(), usr=usr, pwd=pwd)

In [ ]:
#sei_browser.itera_processos()

In [9]:
sis = sistemas.Sis(page.webdriver.Ie(), usr, pwd, 'cidadao')

AttributeError: 'bool' object has no attribute 'send_keys'

In [ ]:
import pandas as pd

df = pd.re

In [ ]:
sis.consulta('01711253304', '604', 'cpf')